# Quaternions representing rotations SO(3)
This notebook is a short sumamry of how SO(3) is handled in Pinocchio, and mostly quaternion representation. 
## SO(3), Euler angles, rotations

In [ ]:
import pinocchio
from pinocchio import SE3, Quaternion
import numpy as np
from numpy.linalg import norm
M = SE3.Random()
R = M.rotation
p = M.translation
print('R =\n' + str(R))
print('p =\n' + str(p.T))

A rotation is simply a 3x3 matrix. It has a unit norm:

In [ ]:
print(R @ R.T)

It can be equivalently represented by a quaternion. Here we have made the choice to create a specific class for quaternions (i.e. they are not vectors, and can be e.g. multiplied), but you can get the 4 coefficients with the adequate method. Note that the corresponding vector is also of norm 1.

In [ ]:
quat = Quaternion(R)
print(norm(quat.coeffs()))

Angle-axis representation are also implemented in the class AngleAxis.

In [ ]:
from pinocchio import AngleAxis
utheta = AngleAxis(quat)
print(utheta.angle, utheta.axis)

You can display rotation in the viewer.

In [ ]:
from tp2.meshcat_viewer_wrapper import MeshcatVisualizer
viz = MeshcatVisualizer()
viz.viewer.jupyter_cell()

In [ ]:
viz.addBox('world/box', [.1, .2, .3], [1, 0, 0, 1])
viz.applyConfiguration('world/box', [.1, .2, .3] + list(quat.coeffs()))

### Quaternion you said?
Quaternions are "complex of complex", introduced form complex as complex are from reals. Let's try to understand what they contains in practice. Quaternions are of the form w+xi+yj+zk, with w,x,y,z real values, and i,j,k the 3 imaginary numbers. We store them as 4-d vectors, with the real part first: quat = [x,y,z,w]. We can interprete w as encoding the cosinus of the rotation angle. Let's see that.

In [ ]:
from numpy import arccos
print(arccos(quat[3]))
print(AngleAxis(quat).angle)

Indeed, w = cos(θ/2). Why divided by two? For that, let's see how the quaternion can be used to represent a rotation. We can encode a 3D vector in the imaginary part of a quaternion.

In [ ]:
p = np.random.rand(3)
qp = Quaternion(0., p[0], p[1], p[2])
print(qp)

The real product extends over quaternions, so let's try to multiply quat with p:

In [ ]:
print(quat * qp)

Well that's not a pure imaginary quaternion anymore. And the imaginary part does not contains somethig that looks like the rotated point:

In [ ]:
print(quat.matrix() @ p)

The pure quaternion is obtained by multiplying again on the left by the conjugate (w,-x,-y,-z).

In [ ]:
print(quat * qp * quat.conjugate())

That is a pure quaternion, hence encoding a point, and does corresponds to R*p. Magic, is it not? We can prove that the double product of quaternion does corresponds to the rotation. Indeed, a quaternion rather encode an action (a rotation) in $R^4$, but which moves our point p outside of $R^3$. The conjugate rotation brings it back in $R^3$ but applies a second rotation. Since we rotate twice, it is necessary to apply only half of the angle each time.
What if we try to apply the rotation quat on the imaginary part of the quaternion?

In [ ]:
qim = Quaternion(quat)  # copy
qim[3] = 0
print(qim, quat * qim * quat.conjugate())

What kind of conclusion can we get from this? What geometrical interpretation can we give to $q_{im}$? What about $||q_{im}||$?

### The SLERP example
Let's practice! Implement a linear interpolation between two position p0 and p1, i.e. find the position p(t) with t varying from 0 to 1, with p(0)=p0, p(1)=p1 and continuity between the two extrema.

In [ ]:
%do_not_load appendix/solution_lerp.py

LERP with quaternions is not working because they are not normalize. Instead we can take either the normalization of the LERP (NLERP), or the spherical LERP (SLERP). 

In [ ]:
%do_not_load appendix/solution_slerp.py

## Cheat sheet

You can convert quaternion to rotation matrix and create SE3 objects as follows:


In [ ]:
qu = Quaternion(.7,.2,.2,.6)# Quaternion: take care that norm <= 1 (and approx 1)
R  = qu.matrix()                # Create a rotation matrix from quaternion
p  = np.array([0.,0.,0.77])     # Translation (R3) vector)
M  = SE3(R,p)                   # Create a nomogeneous matrix from R,P

# Typical tool position
from pinocchio.utils import rotate
M = SE3(rotate('z',1.)@rotate('x',.2), np.array([0.1,0.02,.65]))